In [1]:
DEV_PATH = '/home/frossi/dev/'
import sys
if not DEV_PATH in sys.path:
    sys.path.append(DEV_PATH)

from mavis import *

mp.set_start_method('fork')

In [2]:
%%time
cartPointingCoords = np.asarray([[5,5]])
NGS_flux = [10000, 30000, 5000]
NGS_SR_1650 = [0.4, 0.2, 0.6]
NGS_FWHM_mas = [90, 110, 85]
polarNGSCoords = np.asarray([[30.0,0.0], [50.0,100.0],[10.0,240.0]])
cartNGSCoords = np.asarray([polarToCartesian(polarNGSCoords[0,:]), polarToCartesian(polarNGSCoords[1,:]), polarToCartesian(polarNGSCoords[2,:])])
Ctot = computeTotalResidualMatrix(cartPointingCoords, cartNGSCoords, NGS_flux, NGS_SR_1650, NGS_FWHM_mas, True)
print('Ctot', Ctot)

Ctot [[[12145.83654855 -2405.86006388]
  [-2405.86006388 14191.5529846 ]]]
CPU times: user 4.4 s, sys: 784 ms, total: 5.18 s
Wall time: 5.18 s


In [4]:
Ctot = np.asarray(Ctot)
cov_ellipses = ellipsesFromCovMats(Ctot)
print(cov_ellipses)
print( [ (cov_ellipses[i,1]*0.001, cov_ellipses[i,2]*0.001, cov_ellipses[i,0]*180/np.pi) for i in range(1)])

[[-5.84398118e-01  1.05544259e+04  1.57829636e+04]]
[(10.55442592402095, 15.782963609123778, -33.48354574357587)]


In [3]:
%%time
xx, yy = np.linspace(-15,15,7), np.linspace(-15,15,7)
cartPointingCoords =  np.dstack(np.meshgrid(xx, yy)).reshape(-1, 2)
NGS_flux = [10000, 30000, 5000]
NGS_SR_1650 = [0.4, 0.2, 0.6]
NGS_FWHM_mas = [90, 110, 85]
polarNGSCoords = np.asarray([[30.0,0.0], [50.0,100.0],[10.0,240.0]])
cartNGSCoords = np.asarray([polarToCartesian(polarNGSCoords[0,:]), polarToCartesian(polarNGSCoords[1,:]), polarToCartesian(polarNGSCoords[2,:])])
Ctot = computeTotalResidualMatrix(cartPointingCoords, cartNGSCoords, NGS_flux, NGS_SR_1650, NGS_FWHM_mas)
# print('Ctot', Ctot)

CPU times: user 2.6 s, sys: 852 ms, total: 3.46 s
Wall time: 15 s


In [4]:
%matplotlib ipympl
Ctot = np.asarray(Ctot)
cov_ellipses = ellipsesFromCovMats(Ctot)
plotEllipses(cartPointingCoords, cov_ellipses, 1e-4)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
%%timeit
Ctot = computeTotalResidualMatrix(cartPointingCoords, cartNGSCoords, NGS_flux, NGS_SR_1650, NGS_FWHM_mas)

In [7]:
rh1 = mf['ZernikeCovarianceD'].rhs
lh1 = mf['ZernikeCovarianceD'].lhs
display(lh1)
display(rh1)

dW_phi(rho)

0.0229*(-1)**m_k*2**(-0.5*KroneckerDelta(0, m_j) - 0.5*KroneckerDelta(0, m_k) + 1)*I**(n_j + n_k)*sqrt((n_j + 1)*(n_k + 1))*(I**(3*m_j + 3*m_k)*cos(theta*(m_j + m_k) + pi*((1 - KroneckerDelta(0, m_j))*((-1)**j - 1) + (1 - KroneckerDelta(0, m_k))*((-1)**k - 1))/4)*besselj(m_j + m_k, 2*pi*f*h*rho) + I**(3*Abs(m_j - m_k))*cos(theta*(m_j - m_k) + pi*((1 - KroneckerDelta(0, m_j))*((-1)**j - 1) - (1 - KroneckerDelta(0, m_k))*((-1)**k - 1))/4)*besselj(Abs(m_j - m_k), 2*pi*f*h*rho))*besselj(n_j + 1, 2*pi*R_1*f)*besselj(n_j + 1, 2*pi*R_2*f)/(pi*R_1*R_2*f*r_0**(5/3)*((1/L_0)**2 + f**2)**(11/6))

In [10]:
# Noll indices: i,j = 2,3 : OK
# L0 = 22.0 : OK
# r_0 = calcolare da Cn2 (0.1) # OK
# h = 10000 : OK 
# rho max: 120 asec (convertito in radianti)
# theta= pi/4 : OK
# f = [1e-3, 1] : OK
paramsDict = {'theta': np.pi/4.0, 'L_0': 22.0, 'r_0': 0.1, 'R_1': TelescopeDiameter/2.0, 'R_2': TelescopeDiameter/2.0, 'h': DM_height }
#, 'f_min': 1e-3, 'f_max': 1e3
_integrationLimits = (sp.symbols('f', positive=True) , 1e-3, 10)

_integrand1 = subsParamsByName(cov_expr_jk(rh1, 2, 2), paramsDict)
display(_integrand1)

mItCPU = Integrator('none', np, np.float64)
# usando GPU ho un valore costante: debug: solved!
mItGPU = Integrator('none', cp, cp.float64)
mIt = mItGPU

fig, ax = plt.subplots(figsize=(10,5))
plt.xscale('linear')
plt.yscale('linear')
_integral1 = sp.Integral(_integrand1, _integrationLimits)
display(_integral1)
xplot1, zplot1 = mIt.IntegralEval(lh1, _integral1, [('rho',  0.00001, 120.0/ 206265.0, 13, 'linear')], [(1025, 'linear')], method='rect')
xx, zz = cp.asnumpy(xplot1[0]), cp.asnumpy(zplot1)
ax.plot(xx, zz )
hdul = fits.open('data/ex_covar.fits')
cv_plot = np.asarray(hdul[0].data, np.float64)
ax.plot(xx, cv_plot)
plt.show()

#_lhs = sp.symbols('C_v', positive=True)
#_rhs = _integrand1
#_eq = sp.Eq(_lhs, _rhs)
#myF = Formulary('local', ['ZernikeCoval'], [(_lhs, _rhs, _eq)])
#myF.plotFormula('ZernikeCoval',
#            {'rho': 60.0/ 206265.0},
#            ['f'],
#            [ np.asarray(np.geomspace(1e-3, 10, 1024*2), dtype=np.float64) ],
#            ["scipy", pplib], False)

0.265730960724332*(besselj(0, 20000*pi*f*rho) - 6.12323399573677e-17*besselj(2, 20000*pi*f*rho))*besselj(2, 8.0*pi*f)**2/(pi*f*(f**2 + 0.00206611570247934)**(11/6))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Integral(0.265730960724332*(besselj(0, 20000*pi*f*rho) - 6.12323399573677e-17*besselj(2, 20000*pi*f*rho))*besselj(2, 8.0*pi*f)**2/(pi*f*(f**2 + 0.00206611570247934)**(11/6)), (f, 0.001, 10))